#### MultiSearch Agent RAG

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI

In [8]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [3]:
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

USER_AGENT environment variable not set, consider setting it to identify your requests.


##### Create Tools

In [4]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv_api_wrapper = ArxivAPIWrapper()
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

##### Store documents in vector DB

In [5]:
loader = WebBaseLoader("https://docs.smith.langchain.com/concepts/evaluation")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()

##### Create Retrival Tool

In [27]:
retrival_tool= create_retriever_tool(retriever,"langsmith_search",description="Search for information about LangSmith eval documentation. For any questions about LangSmith, you must use this tool!")

In [28]:
tools=[wikipedia,arxiv, retrival_tool]

In [29]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [30]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [31]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [32]:
agent_executor.invoke({"input":"How to create dataset in langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'create dataset'}`


This can be a useful way to get a lot of datapoints, quickly.tipTo learn more about creating datasets in LangSmith, see our LangSmith Evaluation series:See our video on Manually curated datasets.See our videos on Datasets from tracesTypes of datasetsâ€‹LangSmith offers three distinct dataset types:kv (Key-Value) Dataset:"Inputs" and "outputs" are represented as arbitrary key-value pairs.The kv dataset is the most versatile and default type, suitable for a wide range of evaluation scenarios.This dataset type is ideal for evaluating chains and agents that require multiple inputs or generate multiple outputs.llm (Language Model) Dataset:The llm dataset is designed for evaluating "completion" style language models.The "inputs" dictionary contains a single "input" key mapped to the prompt string.The "outputs" dictionary contains a single "output" key mapped to the corresponding respons

{'input': 'How to create dataset in langsmith',
 'output': 'To create a dataset in LangSmith, you can follow these steps:\n\n1. Manually Curated Examples: Start by manually curating examples that represent the inputs and expected outputs for your AI application. This is a recommended way to begin creating datasets.\n\n2. Types of Datasets: LangSmith offers three distinct dataset types:\n   - Key-Value (kv) Dataset: Represents inputs and outputs as key-value pairs, suitable for a wide range of evaluation scenarios.\n   - Language Model (llm) Dataset: Designed for evaluating completion-style language models, with standardized input and output formats.\n   - Synthetic Data: After creating a few hand-crafted examples, you can generate synthetic data to expand your dataset.\n\n3. Collect End User Feedback: Gather feedback from end users to identify areas where your application may not perform well. Incorporate negative feedback into your dataset for future testing.\n\n4. Apply Evaluations: 

In [34]:
agent_executor.invoke({"input":"Who is CM of UP"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Chief Minister of Uttar Pradesh'}`


Page: List of chief ministers of Uttar Pradesh
Summary: The Chief Minister of Uttar Pradesh is the head of the Government of Uttar Pradesh. As per the Constitution of India, the governor is the state's de jure head, but de facto executive authority rests with the chief minister. Following elections to the Uttar Pradesh Legislative Assembly, the governor usually invites the party (or coalition) with a majority of seats to form the government. The governor appoints the chief minister, whose council of ministers are collectively responsible to the assembly. Given that he has the confidence of the assembly, the chief minister's term is for five years and is subject to no term limits.
On 26 January 1950 Govind Ballabh Pant, premier of United Provinces, became the first chief minister of the newly renamed Uttar Pradesh. Including him, 11 out of UP's 21 chief ministers belonged

{'input': 'Who is CM of UP',
 'output': 'Yogi Adityanath of the Bharatiya Janata Party is currently serving as the Chief Minister of Uttar Pradesh since 19 March 2017.'}

In [38]:
agent_executor.invoke({"input":"The Matrix Calculus You Need For Deep Learning"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'The Matrix Calculus You Need For Deep Learning'}`


Published: 2018-07-02
Title: The Matrix Calculus You Need For Deep Learning
Authors: Terence Parr, Jeremy Howard
Summary: This paper is an attempt to explain all the matrix calculus you need in order
to understand the training of deep neural networks. We assume no math knowledge
beyond what you learned in calculus 1, and provide links to help you refresh
the necessary math where needed. Note that you do not need to understand this
material before you start learning to train and use deep learning in practice;
rather, this material is for those who are already familiar with the basics of
neural networks, and wish to deepen their understanding of the underlying math.
Don't worry if you get stuck at some point along the way---just go back and
reread the previous section, and try writing down and working through some
examples. And if you're still stuck, we're happy

{'input': 'The Matrix Calculus You Need For Deep Learning',
 'output': 'The paper titled "The Matrix Calculus You Need For Deep Learning" by Terence Parr and Jeremy Howard provides an explanation of all the matrix calculus necessary to understand the training of deep neural networks. It assumes no math knowledge beyond what you learned in calculus 1 and offers links to refresh the necessary math concepts. This material is intended for those already familiar with the basics of neural networks who wish to deepen their understanding of the underlying math. If you have any questions or need further clarification, you can seek help in the Theory category at forums.fast.ai.'}